In [2]:
# Import Basic libraries.

import pandas as pd
import numpy as np

# Ignore warnings

import warnings

warnings.filterwarnings('ignore')


In [3]:
import pandas as pd

# Load the files and skip any problematic rows.
geolocation_df = pd.read_csv('geolocation.csv', on_bad_lines='skip',sep=';')
market_analysis_df = pd.read_csv('market_analysis.csv', on_bad_lines='skip',sep=';')
market_analysis_2019_df = pd.read_csv('market_analysis_2019.csv', on_bad_lines='skip',sep=';')
amenities_df = pd.read_csv('amenities.csv', on_bad_lines='skip',sep=';')

# Confirm shapes.
print('Loaded all files successfully!\n')
print('Shape of geolocation_df:', geolocation_df.shape)
print('Shape of market_analysis_df:', market_analysis_df.shape)
print('Shape of market_analysis_2019_df:', market_analysis_2019_df.shape)
print('Shape of amenities_df:', amenities_df.shape)

# Check for duplicates.
print('\nChecking for duplicates...')
print('Repeated rows in geolocation_df:', geolocation_df.duplicated().sum())
print('Repeated rows in market_analysis_df:', market_analysis_df.duplicated().sum())
print('Repeated rows in market_analysis_2019_df:', market_analysis_2019_df.duplicated().sum())
print('Repeated rows in amenities_df:', amenities_df.duplicated().sum())

Loaded all files successfully!

Shape of geolocation_df: (49311, 5)
Shape of market_analysis_df: (116619, 14)
Shape of market_analysis_2019_df: (29928, 14)
Shape of amenities_df: (49311, 4)

Checking for duplicates...
Repeated rows in geolocation_df: 0
Repeated rows in market_analysis_df: 0
Repeated rows in market_analysis_2019_df: 0
Repeated rows in amenities_df: 0


In [4]:
# Add 'AIR' Prefix to the unified_id column.

market_analysis_df['unified_id'] = 'AIR' + market_analysis_df['unified_id'].astype(str)

# Head of the table.

market_analysis_df.head()


,unified_id,month,zipcode,city,host_type,bedrooms,bathrooms,guests,revenue,openness,occupancy,nightly rate,lead time,length stay
0,AIR10000347,2020-1,92315,Big Bear Lake,2-5 Units,3,2.0,10,"899,9599915",31,"0,064516129","449,9799957",52,3
1,AIR10000347,2020-10,92315,Big Bear Lake,2-5 Units,3,2.0,10,13950,31,1,450,NaN,NaN
2,AIR10000347,2020-11,92315,Big Bear Lake,2-5 Units,3,2.0,10,13500,30,1,450,NaN,NaN
3,AIR10000347,2020-12,92315,Big Bear Lake,2-5 Units,3,2.0,10,13950,31,1,450,NaN,NaN
4,AIR10000347,2020-2,92315,Big Bear Lake,2-5 Units,3,2.0,10,450,29,"0,034482759",450,8,1


In [5]:
# Concat the market analysis tables by rows.

market_analysis_full_df = pd.concat([market_analysis_df, market_analysis_2019_df], axis=0)

# Set format of month to datetime.

market_analysis_full_df['month'] = pd.to_datetime(market_analysis_full_df['month'], format='%Y-%m')

# Sort by unified_id and month.

market_analysis_full_df = market_analysis_full_df.sort_values(by=['unified_id', 'month']).reset_index(drop=True)

# Head of the table.

market_analysis_full_df.head()

,unified_id,month,zipcode,city,host_type,bedrooms,bathrooms,guests,revenue,openness,occupancy,nightly rate,lead time,length stay
0,AIR10000347,2019-01-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,"13948,75974",31,1,"449,9599915",8,65
1,AIR10000347,2019-02-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,0,28,0,NaN,NaN,NaN
2,AIR10000347,2019-03-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,"899,9199829",31,"0,064516129","449,9599915",8,1
3,AIR10000347,2019-04-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,0,30,0,NaN,NaN,NaN
4,AIR10000347,2019-05-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,0,31,0,NaN,NaN,NaN


In [6]:
# Convert to datetime the month columns amenities and geolocation table.

geolocation_df['month'] = pd.to_datetime(geolocation_df['month'], format='%Y-%m')
amenities_df['month'] = pd.to_datetime(amenities_df['month'], format='%Y-%m')

# Join amenities_df and geolocation_df by 'unified_id'.

amenities_df = amenities_df.merge(geolocation_df, on='unified_id', how='inner')

In [7]:
amenities_df.head()

,unified_id,month_x,hot_tub,pool,month_y,street_name,latitude,longitude
0,AIR10052559,2022-12-01,1,0,2022-12-01,Heavenly Valley Road,"34,23948","-116,87991"
1,AIR10052559,2022-12-01,1,0,2022-11-01,Heavenly Valley Road,"34,23948","-116,87991"
2,AIR10052559,2022-12-01,1,0,2022-10-01,Heavenly Valley Road,"34,23948","-116,87991"
3,AIR10052559,2022-12-01,1,0,2022-09-01,Heavenly Valley Road,"34,23948","-116,87991"
4,AIR10052559,2022-12-01,1,0,2022-07-01,Heavenly Valley Road,"34,23948","-116,87991"


In [8]:
# Drop the rows where month_x and month_y are different.

amenities_df = amenities_df[amenities_df['month_x'] == amenities_df['month_y']].reset_index(drop=True)

# Drop the month_y column.

amenities_df = amenities_df.drop(columns=['month_y'])

# Rename the month_x column to month.

amenities_df = amenities_df.rename(columns={'month_x': 'month'})

In [9]:
amenities_df.head()

,unified_id,month,hot_tub,pool,street_name,latitude,longitude
0,AIR10052559,2022-12-01,1,0,Heavenly Valley Road,"34,23948","-116,87991"
1,AIR10178668,2022-12-01,0,0,Round Drive,"34,25095","-116,94139"
2,AIR10211700,2022-12-01,0,0,NaN,"34,24516","-116,87618"
3,AIR10344705,2022-12-01,0,0,NaN,"34,2675","-116,86089"
4,AIR10424683,2022-12-01,0,1,Starvation Flats Road,"34,25341","-116,88025"


In [10]:
# Order amenities_df by unified_id and month.

amenities_df = amenities_df.sort_values(by=['unified_id', 'month']).reset_index(drop=True)

# Display the first 5 rows of amenities_df.

amenities_df.head()

,unified_id,month,hot_tub,pool,street_name,latitude,longitude
0,AIR10000347,2020-01-01,0,0,Cienega Road,"34,24223354","-116,9432971"
1,AIR10000347,2020-02-01,0,0,Cienega Road,"34,24223354","-116,9432971"
2,AIR10000347,2020-03-01,0,0,Cienega Road,"34,24223354","-116,9432971"
3,AIR10000347,2020-04-01,0,0,Cienega Road,"34,24223354","-116,9432971"
4,AIR10000347,2020-05-01,0,0,Cienega Road,"34,24223354","-116,9432971"


In [11]:
# Join market_analysis_full_df and amenities_df by unified_id and month.

full_df = market_analysis_full_df.merge(amenities_df, on=['unified_id', 'month'], how='left')

# Order full_df by unified_id and month descending.

full_df = full_df.sort_values(by=['unified_id', 'month'], ascending=[True, False]).reset_index(drop=True)

# Display the first 5 rows of full_df.

full_df.head()

,unified_id,month,zipcode,city,host_type,bedrooms,bathrooms,guests,revenue,openness,occupancy,nightly rate,lead time,length stay,hot_tub,pool,street_name,latitude,longitude
0,AIR10000347,2022-08-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,2700,6,1,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AIR10000347,2022-07-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13950,31,1,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AIR10000347,2022-06-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13500,30,1,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AIR10000347,2022-05-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13950,31,1,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AIR10000347,2022-04-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13500,30,1,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# For each unified_id replace all previous NaN values of 'hot_tub, 'pool', 'street_name', 'latitude', 'longitude' with the first non NaN value.

# Fill hot_tub
full_df['hot_tub'] = (
    full_df.groupby('unified_id')['hot_tub']
           .apply(lambda group: group.ffill().bfill())
           .reset_index(drop=True)
)

# Fill pool
full_df['pool'] = (
    full_df.groupby('unified_id')['pool']
           .apply(lambda group: group.ffill().bfill())
           .reset_index(drop=True)
)

# Fill street_name
full_df['street_name'] = (
    full_df.groupby('unified_id')['street_name']
           .apply(lambda group: group.ffill().bfill())
           .reset_index(drop=True)
)

# Fill latitude
full_df['latitude'] = (
    full_df.groupby('unified_id')['latitude']
           .apply(lambda group: group.ffill().bfill())
           .reset_index(drop=True)
)

# Fill longitude
full_df['longitude'] = (
    full_df.groupby('unified_id')['longitude']
           .apply(lambda group: group.ffill().bfill())
           .reset_index(drop=True)
)

In [13]:
full_df.head()

,unified_id,month,zipcode,city,host_type,bedrooms,bathrooms,guests,revenue,openness,occupancy,nightly rate,lead time,length stay,hot_tub,pool,street_name,latitude,longitude
0,AIR10000347,2022-08-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,2700,6,1,450,NaN,NaN,0.0,0.0,Cienega Road,"34,24223","-116,9433"
1,AIR10000347,2022-07-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13950,31,1,450,NaN,NaN,0.0,0.0,Cienega Road,"34,24223","-116,9433"
2,AIR10000347,2022-06-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13500,30,1,450,NaN,NaN,0.0,0.0,Cienega Road,"34,24223","-116,9433"
3,AIR10000347,2022-05-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13950,31,1,450,NaN,NaN,0.0,0.0,Cienega Road,"34,24223","-116,9433"
4,AIR10000347,2022-04-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,13500,30,1,450,NaN,NaN,0.0,0.0,Cienega Road,"34,24223","-116,9433"


In [14]:
# Export as csv the full_df. without index.

full_df.to_csv('Airbnb new.csv', index=False)

In [15]:
import pandas as pd

# Load the files and skip any problematic rows.
zip_92252 = pd.read_csv('sales_properties_total_zipcode_92252.csv', on_bad_lines='skip',sep=';')
zip_pool_92252 = pd.read_csv('sales_properties_with_pool_zipcode_92252.csv', on_bad_lines='skip',sep=';')
zip_92284 = pd.read_csv('sales_properties_total_zipcode_92284.csv', on_bad_lines='skip',sep=';')
zip_pool_92284 = pd.read_csv('sales_properties_with_pool_zipcode_92284.csv', on_bad_lines='skip',sep=';')
zip_92314 = pd.read_csv('sales_properties_total_zipcode_92314.csv' , on_bad_lines='skip',sep=';') 
zip_92315 = pd.read_csv('sales_properties_total_zipcode_92315.csv', on_bad_lines='skip',sep=';')

# Shape of the tables.

print('Shape of zip_92252: ', zip_92252.shape)
print('Shape of zip_pool_92252: ', zip_pool_92252.shape)
print('Shape of zip_92284: ', zip_92284.shape)
print('Shape of zip_pool_92284: ', zip_pool_92284.shape)
print('Shape of zip_92314: ', zip_92314.shape)
print('Shape of zip_92315: ', zip_92315.shape)

# Optional: Check for duplicates.
print('\nChecking for duplicates...')
print('Repeated files in zip_92252: ', zip_92252.duplicated().sum())
print('Repeated files in zip_pool_92252: ', zip_pool_92252.duplicated().sum())
print('Repeated files in zip_92284: ', zip_92284.duplicated().sum())
print('Repeated files in zip_pool_92284: ', zip_pool_92284.duplicated().sum())
print('Repeated files in zip_92314: ', zip_92314.duplicated().sum())
print('Repeated files in zip_92315: ', zip_92315.duplicated().sum())

Shape of zip_92252:  (73, 23)
Shape of zip_pool_92252:  (15, 23)
Shape of zip_92284:  (117, 23)
Shape of zip_pool_92284:  (15, 23)
Shape of zip_92314:  (57, 23)
Shape of zip_92315:  (106, 23)

Checking for duplicates...
Repeated files in zip_92252:  0
Repeated files in zip_pool_92252:  0
Repeated files in zip_92284:  0
Repeated files in zip_pool_92284:  0
Repeated files in zip_92314:  0
Repeated files in zip_92315:  0


In [16]:
# Create a column 'pool' with value 1 in 'zip_pool_92252' and in 'zip_pool_92284'.

zip_pool_92252['pool'] = 1

zip_pool_92284['pool'] = 1

In [17]:
# Select the column 'pool' and 'Url'.

zip_pool_92252 = zip_pool_92252[['pool', 'Url']]
zip_pool_92284 = zip_pool_92284[['pool', 'Url']]

# Merge 'zip_92252' and 'zip_pool_92252' by 'Url'.

zip_92252 = zip_92252.merge(zip_pool_92252, on='Url', how='left')

# Merge 'zip_92284' and 'zip_pool_92284' by 'Url'.

zip_92284 = zip_92284.merge(zip_pool_92284, on='Url', how='left')

# Fill the NaN values of 'pool' with 0.

zip_92252['pool'] = zip_92252['pool'].fillna(0)
zip_92284['pool'] = zip_92284['pool'].fillna(0)

# Convert 'pool' to int.

zip_92252['pool'] = zip_92252['pool'].astype(int)
zip_92284['pool'] = zip_92284['pool'].astype(int)

In [18]:
# Create a column 'pool' with value 0 in 'zip_92314' and 'zip_92315'.

zip_92314['pool'] = 0
zip_92315['pool'] = 0

# Concatenate 'zip_92252', 'zip_92284', 'zip_92314' and 'zip_92315'.

zip_df = pd.concat([zip_92252, zip_92284, zip_92314, zip_92315], ignore_index=True)

# Display the first 5 rows of 'zip_df'.

zip_df.head()

,Url,Zestimate,Price,Rent Zestimate,Days On Zillow,Bathrooms,Bedrooms,Living Area,Lot Size,Home Type,...,Country,Broker Name,Has 3D Model,Has Image,Has Video,isZillowOwned,sgapt,statusText,statusType,pool
0,https://www.zillow.com/homedetails/8198-Uphill...,NaN,18000000,2435.0,Not specified,4,3,5470,NaN,SINGLE_FAMILY,...,USA,AKGRE Inc,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,0
1,https://www.zillow.com/homedetails/4757-Avenid...,223200.0,399990,1786.0,Not specified,2,3,1056,NaN,SINGLE_FAMILY,...,USA,Champion Realty,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,0
2,https://www.zillow.com/homedetails/61018-Presc...,568947.0,575000,2300.0,Not specified,2,3,1577,NaN,SINGLE_FAMILY,...,USA,BHG Desert Lifestyle Properties,VERDADERO,VERDADERO,VERDADERO,FALSO,For Sale (Broker),House for sale,FOR_SALE,0
3,https://www.zillow.com/homedetails/61875-Anaco...,851656.0,899000,4798.0,Not specified,2,3,1434,NaN,SINGLE_FAMILY,...,USA,Cherie Miller & Assoc.,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,0
4,https://www.zillow.com/homedetails/61879-Petun...,NaN,369900,1825.0,Not specified,1,3,1040,NaN,SINGLE_FAMILY,...,USA,Cherie Miller & Assoc.,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,0


In [19]:
# Export as csv the zip_df. withouth index.

zip_df.to_csv('Zillow.csv', index=False)

In [20]:
# Rename the column "zip" to "zipcode".
zip_df = zip_df.rename(columns={'zip': 'zipcode'})

In [53]:
# Convert all column names to lowercase for both DataFrames.
full_df.columns = full_df.columns.str.lower()
zip_df.columns = zip_df.columns.str.lower()

In [54]:
# Rename zip to zipcode.
zip_df.rename(columns={'zip': 'zipcode'}, inplace=True)
zip_df.columns

Index(['url', 'zestimate', 'price', 'rent zestimate', 'days on zillow',
       'bathrooms', 'bedrooms', 'living area', 'lot size', 'home type',
       'street address', 'city', 'zipcode', 'state', 'country', 'broker name',
       'has 3d model', 'has image', 'has video', 'iszillowowned', 'sgapt',
       'statustext', 'statustype', 'pool'],
      dtype='object')

In [55]:
# merging the two files(full_df and zip_df).
compelete_df = pd.merge(
    full_df,
    zip_df,
    on=['zipcode', 'bedrooms', 'city', 'pool','bathrooms'],  # multiple matching conditions
    how='inner'  # only matching rows
)

In [65]:
compelete_df.head()

,unified_id,month,zipcode,city,host_type,bedrooms,bathrooms,guests,revenue,openness,...,broker name,has 3d model,has image,has video,iszillowowned,sgapt,statustext,statustype,lead_time,length_of_stay
0,AIR10000347,2022-08-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,2700,6,...,Re/Max Big Bear,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,NaN,NaN
1,AIR10000347,2022-08-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,2700,6,...,PONCE & PONCE REALTY,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,NaN,NaN
2,AIR10000347,2022-08-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,2700,6,...,Re/Max Big Bear,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,NaN,NaN
3,AIR10000347,2022-08-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,2700,6,...,VACASA REAL ESTATE,FALSO,VERDADERO,FALSO,FALSO,For Sale (Broker),House for sale,FOR_SALE,NaN,NaN
4,AIR10000347,2022-08-01,92315,Big Bear Lake,2-5 Units,3,2.0,10,2700,6,...,Re/Max Big Bear,FALSO,VERDADERO,FALSO,FALSO,New Construction,New construction,FOR_SALE,NaN,NaN


In [57]:
# Checking unique values of the column name (City).
unique_values = compelete_df['city'].unique().tolist()
print(unique_values)

['Big Bear Lake', 'Joshua Tree', 'Big Bear City', 'Yucca Valley']


In [58]:
# Replace commas with dots and convert to float.
compelete_df['lead_time'] = compelete_df['lead time'].astype(str).str.replace(',', '.').astype(float).round(1)
compelete_df['length_of_stay'] = compelete_df['length stay'].astype(str).str.replace(',', '.').astype(float).round(1)
compelete_df['occupancy'] = compelete_df['occupancy'].astype(str).str.replace(',', '.').astype(float).round(1)

In [59]:
compelete_df['nightly rate'] =compelete_df['nightly rate'].astype(str).str.replace(',', '.').astype(float).round(2)

In [60]:
compelete_df['latitude'] = compelete_df['latitude'].astype(str).str.replace(',', '.')
compelete_df['longitude'] = compelete_df['longitude'].astype(str).str.replace(',', '.')
compelete_df['revenue'] = compelete_df['revenue'].astype(str).str.replace(',', '.')

In [61]:
compelete_df.shape[0]

1285313

In [62]:
# Export as csv the zip_df. withouth index.

compelete_df.to_csv('Final_55 Codar_final_cleaned_data.csv', index=False)

In [63]:
len(compelete_df['unified_id'])

1285313